In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score, classification_report, plot_confusion_matrix, precision_score, roc_auc_score
from sklearn.pipeline import Pipeline

In [2]:
master_df = pd.read_csv('SAMPLE2.csv')

In [3]:
master_df.head()

,Primary Key,Created,Mileage In,Work Order,Model,Line Type,Service Category,Service Package,Description,Definition,...,Quantity,Total,Discount,Net Total,ContactID,ServiceItemID,InvoiceID,labels,Make,Year
0,1,12/28/2018 10:18,33320,49,civic,Material,"Oil, Lube & Preventive Maintenance Service",MINOR Preventive Maintenance Service (MEDIUM S...,OIL FILTER HONDA,Engine Oil Filter,...,1.0,696.43,0.0,696.43,a271834a-1afc-4e62-8c71-638025174c1f,ff01e2a4-d2d5-48ff-8ccc-d5605a653df4,99833bf9-865b-4e2c-9bd1-a0ed0cebea8e,preventive maintenance,honda,2012
1,2,12/28/2018 10:18,33320,49,civic,Material,"Oil, Lube & Preventive Maintenance Service",MINOR Preventive Maintenance Service (MEDIUM S...,ENGINE FLUSHING,NaN,...,1.0,142.86,0.0,142.86,a271834a-1afc-4e62-8c71-638025174c1f,ff01e2a4-d2d5-48ff-8ccc-d5605a653df4,99833bf9-865b-4e2c-9bd1-a0ed0cebea8e,preventive maintenance,honda,2012
2,3,12/28/2018 10:18,33320,49,civic,Material,"Oil, Lube & Preventive Maintenance Service",MINOR Preventive Maintenance Service (MEDIUM S...,BRAKE CLEANER 14oz,NaN,...,1.0,227.68,0.0,227.68,a271834a-1afc-4e62-8c71-638025174c1f,ff01e2a4-d2d5-48ff-8ccc-d5605a653df4,99833bf9-865b-4e2c-9bd1-a0ed0cebea8e,preventive maintenance,honda,2012
3,4,12/28/2018 10:18,33320,49,civic,Labor,"Oil, Lube & Preventive Maintenance Service",MINOR Preventive Maintenance Service (MEDIUM S...,Labor,NaN,...,3.0,1339.29,0.0,1339.29,a271834a-1afc-4e62-8c71-638025174c1f,ff01e2a4-d2d5-48ff-8ccc-d5605a653df4,99833bf9-865b-4e2c-9bd1-a0ed0cebea8e,preventive maintenance,honda,2012
4,5,12/28/2018 10:18,33320,49,civic,Material,"Oil, Lube & Preventive Maintenance Service",MINOR Preventive Maintenance Service (MEDIUM S...,REPSOL ELITE INJECTION 10W40 1L - GAS/DIESEL,Engine Oil,...,4.0,1178.56,0.0,1178.56,a271834a-1afc-4e62-8c71-638025174c1f,ff01e2a4-d2d5-48ff-8ccc-d5605a653df4,99833bf9-865b-4e2c-9bd1-a0ed0cebea8e,preventive maintenance,honda,2012


In [4]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37548 entries, 0 to 37547
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Primary Key       37548 non-null  int64  
 1   Created           37548 non-null  object 
 2   Mileage In        37548 non-null  int64  
 3   Work Order        37548 non-null  int64  
 4   Model             37548 non-null  object 
 5   Line Type         37548 non-null  object 
 6   Service Category  37548 non-null  object 
 7   Service Package   37525 non-null  object 
 8   Description       37548 non-null  object 
 9   Definition        15582 non-null  object 
 10  Price             37548 non-null  float64
 11  Quantity          37548 non-null  float64
 12  Total             37548 non-null  float64
 13  Discount          37548 non-null  float64
 14  Net Total         37548 non-null  float64
 15  ContactID         37548 non-null  object 
 16  ServiceItemID     37548 non-null  object

In [5]:
master_df['labels'].value_counts()

preventive maintenance                          15899
change oil, lube and flushing                    4957
complete vehicle inspection                      2799
steering and wheel alignment                     1962
Brakes                                           1590
suspension                                       1252
Air Conditioner General Cleaning Service         1131
Ignition and fuel general services                988
clutch general services                           978
Engine                                            928
tire general services and repair                  851
electrical general services and replacement       625
Valve Cover Gasket                                545
Cooling Repair, charging and replacement          515
Transmission general services                     430
Vacuum/Charging Service                           369
Radiator general services                         349
Wheel Bearing                                     230
Drive train general services

In [6]:
# car labels; getting only the entries with greater than 250
label_list = master_df['labels'].value_counts().to_frame()
label_list = label_list[label_list['labels'] <= 140]
label_list = list(label_list.index)
label_list

['Other Accessories Services',
 'Door Servicing',
 'Counter / Cash Sale',
 'Body Repair and Paint Works',
 'Accessories General Service',
 'Other Sublet Services (Machine Shop)',
 'Car Horn Repair & Services',
 'Car Hood Services',
 'Towing',
 'Exhaust General Service',
 'electrical general services',
 'Car Seat Services']

In [7]:
master_df = master_df[~master_df['labels'].isin(label_list)]

In [8]:
# car make; getting only the entries with greater than 100 
car_list = master_df['Make'].value_counts().to_frame()
car_list = car_list[car_list['Make'] <= 100]
car_list = list(car_list.index)
car_list

['audi',
 'mercedes benz',
 'lexus',
 'volvo',
 'volkswagen',
 'dodge',
 'lincoln',
 'foton',
 'chrysler',
 'peugeot',
 'jeep',
 'mg']

In [9]:
master_df = master_df[~master_df['Make'].isin(car_list)]

In [10]:
master_df = master_df[master_df['Service Category'] != 'Accessories (Gauges, Power Accessories, Vision & Air Bag)']

In [11]:
master_df = master_df[master_df['labels'] != 'preventive maintenance']
master_df = master_df[master_df['labels'] != 'Engine Detailing']

# 
master_df = master_df[master_df['labels'] != 'Drive train general services and replacement']
master_df = master_df[master_df['labels'] != 'filter and sensor (air and fuel)']

In [12]:
labl = master_df['labels'].value_counts().to_frame()
labl

,labels
"change oil, lube and flushing",4924
complete vehicle inspection,2766
steering and wheel alignment,1930
Brakes,1575
suspension,1238
Air Conditioner General Cleaning Service,1103
clutch general services,978
Ignition and fuel general services,941
Engine,918
tire general services and repair,840


In [13]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20397 entries, 13 to 37547
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Primary Key       20397 non-null  int64  
 1   Created           20397 non-null  object 
 2   Mileage In        20397 non-null  int64  
 3   Work Order        20397 non-null  int64  
 4   Model             20397 non-null  object 
 5   Line Type         20397 non-null  object 
 6   Service Category  20397 non-null  object 
 7   Service Package   20375 non-null  object 
 8   Description       20397 non-null  object 
 9   Definition        7957 non-null   object 
 10  Price             20397 non-null  float64
 11  Quantity          20397 non-null  float64
 12  Total             20397 non-null  float64
 13  Discount          20397 non-null  float64
 14  Net Total         20397 non-null  float64
 15  ContactID         20397 non-null  object 
 16  ServiceItemID     20397 non-null  objec

In [14]:
X = master_df[['Year', 'Make', 'Model','Mileage In']]
Y = master_df['labels']

In [15]:
# category encoding
# !pip install category_encoders

import category_encoders as ce
encoder = ce.BinaryEncoder(cols=['Year', 'Make', 'Model'], return_df = True)

In [16]:
encoder.fit(X)

BinaryEncoder(cols=['Year', 'Make', 'Model'])

# Splitting of Dataset

In [17]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state = 4244, test_size = 0.20, stratify = Y)

In [18]:
X_train_encoded = encoder.transform(x_train)
X_test_encoded = encoder.transform(x_test)

In [19]:
print(X_train_encoded.shape, X_test_encoded.shape)

(16317, 22) (4080, 22)


# Random Forest Classifier

In [20]:
model_ = RandomForestClassifier(random_state = 4244)

In [21]:
pipeline = Pipeline([('model', model_)])
pipeline.fit(X_train_encoded, y_train.values.ravel())

Pipeline(steps=[('model', RandomForestClassifier(random_state=4244))])

In [22]:
y_pred = pipeline.predict(X_test_encoded)

print("Precision score: {:.3f}".format(precision_score(y_test,y_pred , average='macro')))
print("Accuracy Score:{:.3f}".format(accuracy_score(y_test, y_pred)))
print("Classification Report: \n", classification_report(y_test, y_pred))

Precision score: 0.642
Accuracy Score:0.702
Classification Report: 
                                              precision    recall  f1-score   support

   Air Conditioner General Cleaning Service       0.71      0.81      0.76       221
    Alternator Diagnosis / Component Repair       0.59      0.55      0.57        29
                                     Brakes       0.63      0.64      0.64       315
   Cooling Repair, charging and replacement       0.65      0.65      0.65       103
                                     Engine       0.62      0.67      0.65       184
         Ignition and fuel general services       0.60      0.48      0.53       188
                  Radiator general services       0.59      0.57      0.58        70
              Transmission general services       0.75      0.59      0.66        85
                    Vacuum/Charging Service       0.58      0.56      0.57        72
                         Valve Cover Gasket       0.49      0.53      0.51      

In [23]:
# fig, ax = plt.subplots(figsize = (15,15))
# plot_confusion_matrix(pipeline, X_test_encoded, y_test, ax = ax, cmap='Greys')
# plt.xticks(rotation=90)

# AdaBoost Classifier

In [24]:
model_ = AdaBoostClassifier(base_estimator = RandomForestClassifier(), random_state = 4244)

In [25]:
pipeline = Pipeline([('model', model_)])
pipeline.fit(X_train_encoded, y_train.values.ravel())

Pipeline(steps=[('model',
                 AdaBoostClassifier(base_estimator=RandomForestClassifier(),
                                    random_state=4244))])

In [26]:
y_pred = pipeline.predict(X_test_encoded)

print("Precision score: {:.3f}".format(precision_score(y_test,y_pred , average='macro')))
print("Accuracy Score:{:.3f}".format(accuracy_score(y_test, y_pred)))
print("Classification Report: \n", classification_report(y_test, y_pred))

Precision score: 0.644
Accuracy Score:0.695
Classification Report: 
                                              precision    recall  f1-score   support

   Air Conditioner General Cleaning Service       0.72      0.81      0.76       221
    Alternator Diagnosis / Component Repair       0.67      0.48      0.56        29
                                     Brakes       0.65      0.65      0.65       315
   Cooling Repair, charging and replacement       0.65      0.60      0.62       103
                                     Engine       0.67      0.64      0.65       184
         Ignition and fuel general services       0.59      0.48      0.53       188
                  Radiator general services       0.61      0.50      0.55        70
              Transmission general services       0.75      0.52      0.61        85
                    Vacuum/Charging Service       0.56      0.56      0.56        72
                         Valve Cover Gasket       0.49      0.44      0.47      

In [27]:
# fig, ax = plt.subplots(figsize = (15,15))
# plot_confusion_matrix(pipeline, X_test_encoded, y_test, ax = ax, cmap='Greys')
# plt.xticks(rotation=90)

# Bagging Classifier

In [28]:
model_ = BaggingClassifier(RandomForestClassifier(), random_state = 4244)

In [29]:
pipeline = Pipeline([('model', model_)])

pipeline.fit(X_train_encoded, y_train.values.ravel())

Pipeline(steps=[('model',
                 BaggingClassifier(base_estimator=RandomForestClassifier(),
                                   random_state=4244))])

In [30]:
y_pred = pipeline.predict(X_test_encoded)

print("Precision score: {:.3f}".format(precision_score(y_test,y_pred , average='macro')))
print("Accuracy Score:{:.3f}".format(accuracy_score(y_test, y_pred)))
print("Classification Report: \n", classification_report(y_test, y_pred))

Precision score: 0.627
Accuracy Score:0.689
Classification Report: 
                                              precision    recall  f1-score   support

   Air Conditioner General Cleaning Service       0.74      0.80      0.77       221
    Alternator Diagnosis / Component Repair       0.62      0.45      0.52        29
                                     Brakes       0.64      0.65      0.64       315
   Cooling Repair, charging and replacement       0.61      0.57      0.59       103
                                     Engine       0.58      0.67      0.62       184
         Ignition and fuel general services       0.55      0.44      0.49       188
                  Radiator general services       0.60      0.56      0.58        70
              Transmission general services       0.62      0.53      0.57        85
                    Vacuum/Charging Service       0.57      0.56      0.56        72
                         Valve Cover Gasket       0.48      0.54      0.50      

In [31]:
# fig, ax = plt.subplots(figsize = (15,15))
# plot_confusion_matrix(pipeline, X_test_encoded, y_test, ax = ax, cmap='Greys')
# plt.xticks(rotation=90)

# ExtraTreesClassifier

In [32]:
model_ = ExtraTreesClassifier(random_state = 4244)

In [33]:
pipeline = Pipeline([('model', model_)])

pipeline.fit(X_train_encoded, y_train.values.ravel())

Pipeline(steps=[('model', ExtraTreesClassifier(random_state=4244))])

In [34]:
y_pred = pipeline.predict(X_test_encoded)

print("Precision score: {:.3f}".format(precision_score(y_test,y_pred , average='macro')))
print("Accuracy Score:{:.3f}".format(accuracy_score(y_test, y_pred)))
print("Classification Report: \n", classification_report(y_test, y_pred))

Precision score: 0.643
Accuracy Score:0.706
Classification Report: 
                                              precision    recall  f1-score   support

   Air Conditioner General Cleaning Service       0.70      0.84      0.76       221
    Alternator Diagnosis / Component Repair       0.60      0.62      0.61        29
                                     Brakes       0.62      0.69      0.65       315
   Cooling Repair, charging and replacement       0.64      0.68      0.66       103
                                     Engine       0.62      0.74      0.67       184
         Ignition and fuel general services       0.60      0.53      0.56       188
                  Radiator general services       0.60      0.59      0.59        70
              Transmission general services       0.69      0.53      0.60        85
                    Vacuum/Charging Service       0.51      0.56      0.53        72
                         Valve Cover Gasket       0.52      0.49      0.51      

In [35]:
# fig, ax = plt.subplots(figsize = (15,15))
# plot_confusion_matrix(pipeline, X_test_encoded, y_test, ax = ax, cmap='Greys')
# plt.xticks(rotation=90)

# ExtraTreesClassifier with AdaBoostClassifier

In [36]:
model_ = AdaBoostClassifier(base_estimator = ExtraTreesClassifier(), random_state = 4244)

In [37]:
pipeline = Pipeline([('model', model_)])

pipeline.fit(X_train_encoded, y_train.values.ravel())

Pipeline(steps=[('model',
                 AdaBoostClassifier(base_estimator=ExtraTreesClassifier(),
                                    random_state=4244))])

In [38]:
y_pred = pipeline.predict(X_test_encoded)

print("Precision score: {:.3f}".format(precision_score(y_test,y_pred , average='macro')))
print("Accuracy Score:{:.3f}".format(accuracy_score(y_test, y_pred)))
print("Classification Report: \n", classification_report(y_test, y_pred))

Precision score: 0.640
Accuracy Score:0.690
Classification Report: 
                                              precision    recall  f1-score   support

   Air Conditioner General Cleaning Service       0.72      0.77      0.75       221
    Alternator Diagnosis / Component Repair       0.68      0.52      0.59        29
                                     Brakes       0.62      0.66      0.64       315
   Cooling Repair, charging and replacement       0.64      0.61      0.63       103
                                     Engine       0.61      0.64      0.62       184
         Ignition and fuel general services       0.53      0.51      0.52       188
                  Radiator general services       0.61      0.49      0.54        70
              Transmission general services       0.70      0.46      0.55        85
                    Vacuum/Charging Service       0.60      0.51      0.55        72
                         Valve Cover Gasket       0.49      0.54      0.51      

In [39]:
# fig, ax = plt.subplots(figsize = (15,15))
# plot_confusion_matrix(pipeline, X_test_encoded, y_test, ax = ax, cmap='Greys')
# plt.xticks(rotation=90)

# ExtraTreesClassifier with BaggingClassifier

In [40]:
model_ = BaggingClassifier(base_estimator = ExtraTreesClassifier(), random_state = 4244)

In [41]:
pipeline = Pipeline([('model', model_)])

pipeline.fit(X_train_encoded, y_train.values.ravel())

Pipeline(steps=[('model',
                 BaggingClassifier(base_estimator=ExtraTreesClassifier(),
                                   random_state=4244))])

In [42]:
y_pred = pipeline.predict(X_test_encoded)

print("Precision score: {:.3f}".format(precision_score(y_test,y_pred , average='macro')))
print("Accuracy Score:{:.3f}".format(accuracy_score(y_test, y_pred)))
print("Classification Report: \n", classification_report(y_test, y_pred))

Precision score: 0.645
Accuracy Score:0.703
Classification Report: 
                                              precision    recall  f1-score   support

   Air Conditioner General Cleaning Service       0.74      0.81      0.77       221
    Alternator Diagnosis / Component Repair       0.67      0.62      0.64        29
                                     Brakes       0.64      0.67      0.66       315
   Cooling Repair, charging and replacement       0.65      0.64      0.64       103
                                     Engine       0.63      0.73      0.68       184
         Ignition and fuel general services       0.57      0.48      0.52       188
                  Radiator general services       0.61      0.57      0.59        70
              Transmission general services       0.66      0.60      0.63        85
                    Vacuum/Charging Service       0.56      0.56      0.56        72
                         Valve Cover Gasket       0.51      0.55      0.53      

In [43]:
# fig, ax = plt.subplots(figsize = (15,15))
# plot_confusion_matrix(pipeline, X_test_encoded, y_test, ax = ax, cmap='Greys')
# plt.xticks(rotation=90)